In [1]:
from langchain_community.chat_models import ChatOllama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
# search tools
from langchain_community.tools import DuckDuckGoSearchRun

In [2]:

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

In [6]:
search = DuckDuckGoSearchRun()
result = search.invoke("what is bangkok temperature today?")
print(result)

Temperatures in Bangkok fell as low as 59.3 degrees Fahrenheit (15.2 degrees Celsius) on Monday, the lowest the capital has seen in years, as the country's weather agency warned cool conditions ... The Long-range 12 day forecast also includes detail for Bangkok weather today. Live weather reports from Bangkok weather stations and weather warnings that include risk of thunder, high UV index and forecast gales. See the links below the 12-day Bangkok weather forecast table for other cities and towns nearby along with weather conditions for ... Current Bangkok weather condition is Sunny with real-time temperature (24°C), humidity 65%, wind 9.4km/h, pressure (1017mb), UV (0.7), visibility (10km) in Bangkok. ... Today Bangkok. 10 ... Accurate hourly weather forecast for Bangkok, TH including all the relevant parameters like temperature, feels like temperature, wind and gusts, chance of precipitation, and much more from Foreca. Bangkok - Weather forecast from Theweather.com. Weather condition

In [10]:
# Configure Ollama with your local Llama model
model = "llama3.2"  # Adjust to match your local model's name
llm = ChatOllama(model=model, format="json", temperature=0)

tools = [add, multiply, divide, search]

In [ ]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition # this is the checker for the if you got a tool back
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with using search and performing arithmetic on a set of inputs.")

# Node
def reasoner(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Graph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("reasoner", reasoner)
builder.add_node("tools", ToolNode(tools)) # for the tools

# Add edges
builder.add_edge(START, "reasoner")
builder.add_conditional_edges(
    "reasoner",
    # If the latest message (result) from node reasoner is a tool call -> tools_condition routes to tools
    # If the latest message (result) from node reasoner is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "reasoner")
react_graph = builder.compile()

# Display the graph
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

https://github.com/LangGraph-GUI/LangGraph-learn/blob/main/06%20Agent%20Choose%20Tool/main.py

In [11]:
import inspect


In [13]:
signature = inspect.signature(multiply)
docstring = multiply.__doc__ or ""

In [15]:
signature,docstring

(<Signature (a: int, b: int) -> int>,
 'Multiply a and b.\n\n    Args:\n        a: first int\n        b: second int\n    ')

In [16]:
params = [
    {"name": param.name, "type": param.annotation}
    for param in signature.parameters.values()
]
params

[{'name': 'a', 'type': int}, {'name': 'b', 'type': int}]

In [19]:
tool_info = {
    "name": multiply.__name__,
    "description": docstring,
    "parameters": params
}
tool_info

{'name': 'multiply',
 'description': 'Multiply a and b.\n\n    Args:\n        a: first int\n        b: second int\n    ',
 'parameters': [{'name': 'a', 'type': int}, {'name': 'b', 'type': int}]}